In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import math


In [2]:
#constant
TEST_EMBEDDING_FILE = "test_embeddings.pt"
TRAIN_EMBEDDING_FILE = "train_embeddings.pt"
VAL_EMBEDDING_FILE = "val_embeddings.pt"

GNN_TEST_EMBEDDING = f"../GATNN/embeddings/{TEST_EMBEDDING_FILE}"
GNN_TRAIN_EMBEDDING = f"../GATNN/embeddings/{TRAIN_EMBEDDING_FILE}"
GNN_VAL_EMBEDDING = f"../GATNN/embeddings/{VAL_EMBEDDING_FILE}"
BERT_TEST_EMBEDDING = f"../BERT/embedding/{TEST_EMBEDDING_FILE}"
BERT_TRAIN_EMBEDDING = f"../BERT/embedding/{TRAIN_EMBEDDING_FILE}"
BERT_VAL_EMBEDDING = f"../BERT/embedding/{VAL_EMBEDDING_FILE}"



In [3]:
GNN_FILE = "gat_embeddings.pt"
BERT_FILE = "all_embeddings.pt"
GNN_EMBEDDING = f"../GATNN/embeddings/{GNN_FILE}"
BERT_EMBEDDING = f"../BERT/embedding/{BERT_FILE}"

In [4]:
def load_embedding_file(path):
    """
    Hàm load file embedding từ đường dẫn `path`.
    Tự động nhận dạng nhiều kiểu dữ liệu khác nhau:
    - dict có key 'embeddings' hoặc 'emb', 'vectors', 'features'
    - trực tiếp là Tensor
    - list/tuple (chuyển sang Tensor)
    """
    # ---- Bỏ chặn numpy pickle cổ ----
    try:
        torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])
    except Exception:
        pass  # không sao

    # Load thử với weights_only=False ----
    try:
        data = torch.load(path, weights_only=False)
    except Exception as e:
        raise RuntimeError(f"LỖI load file embedding: {e}")
    
    # data = torch.load(path, map_location='cpu')

    #Chuẩn hóa về tensor ----
    # Nếu là tensor trực tiếp
    if isinstance(data, torch.Tensor):
        return {'embeddings': data}
    

    # Nếu là numpy
    if isinstance(data, np.ndarray):
        return {'embeddings': torch.tensor(data)}
    

    # Trường hợp 1: File lưu dạng dict
    if isinstance(data, dict):

        # TH1.1: Dict có key phổ biến 'embeddings'
        if 'embeddings' in data:
            return {
                'embeddings': torch.as_tensor(data['embeddings']),
                'ids': data.get('mol_ids', None)  or data.get('mol_id',None) # có thể không có
            }

        # TH1.2: Một số file embedding dùng key khác
        for key in ['emb', 'vectors', 'features']:
            if key in data:
                return {
                    'embeddings': torch.as_tensor(data[key]),
                    'ids': data.get('mol_ids', None) or data.get('mol_id',None)
                }

        # TH1.3: Dict không rõ cấu trúc → thử convert cả dict sang tensor
        try:
            return {'embeddings': torch.as_tensor(data)}
        except Exception:
            raise ValueError(
                f"Lỗi: Dict trong file {path} có cấu trúc không hỗ trợ để chuyển sang Tensor"
            )

    

    # Trường hợp 3: File là list hoặc tuple → convert sang Tensor
    elif isinstance(data, (list, tuple)):
        try:
            return {'embeddings': torch.as_tensor(data)}
        except Exception:
            raise ValueError(
                f"Lỗi: Không thể chuyển list/tuple trong file {path} sang Tensor"
            )

    # Trường hợp không thuộc loại nào
    else:
        raise ValueError(
            f"Lỗi: Định dạng dữ liệu trong file {path} không được hỗ trợ"
            f"Loại: {type(data)}"
        )


In [6]:
import os
import torch

def check_embedding_file(path):
    """Kiểm tra 1 file embedding và trả về True/False + in thông tin chi tiết."""

    print(f"KIỂM TRA FILE: {path} ======")

    if not os.path.exists(path):
        print("Lỗi: File không tồn tại.")
        return False

    try:
        result = load_embedding_file(path)
        emb = result["embeddings"]

        # --- Kiểm tra embedding có phải Tensor ---
        if not isinstance(emb, torch.Tensor):
            print("Lỗi: embeddings không phải Torch.Tensor.")
            return False

        # --- Kiểm tra số chiều ---
        if emb.ndim < 2:
            print(f"Lỗi: embeddings phải >= 2 chiều, hiện tại: {emb.ndim}")
            return False

        # --- In thông tin hợp lệ ---
        print("✅File hợp lệ!")
        print(f"Kích thước: {tuple(emb.shape)}___Kiểu dữ liệu: {emb.dtype}___Có IDs không? { 'Có' if result.get('ids') is not None else 'Không' }")
       
        return True

    except Exception as e:
        print(f"Lỗi khi load file: {e}")
        return False


def test_all_embeddings():
    """Test toàn bộ file BERT + GNN"""

    bert_files = [
        BERT_TEST_EMBEDDING,
        BERT_TRAIN_EMBEDDING,
        BERT_VAL_EMBEDDING,
        BERT_EMBEDDING
    ]

    gnn_files = [
        GNN_TEST_EMBEDDING,
        GNN_TRAIN_EMBEDDING,
        GNN_VAL_EMBEDDING,
        GNN_EMBEDDING
    ]

   
    print("KIỂM TRA BERT BRANCH==================================")
  

    for f in bert_files:
        check_embedding_file(f)

    
    print("KIỂM TRA GNN BRANCH====================================")

    for f in gnn_files:
        check_embedding_file(f)


if __name__ == "__main__":
    test_all_embeddings()



KIỂM TRA BERT BRANCH==================================
KIỂM TRA FILE: ../BERT/embedding/test_embeddings.pt ======
✅File hợp lệ!
Kích thước: (802, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA FILE: ../BERT/embedding/train_embeddings.pt ======
✅File hợp lệ!
Kích thước: (6411, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA FILE: ../BERT/embedding/val_embeddings.pt ======
✅File hợp lệ!
Kích thước: (801, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA FILE: ../BERT/embedding/all_embeddings.pt ======
✅File hợp lệ!
Kích thước: (8014, 768)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA GNN BRANCH====================================
KIỂM TRA FILE: ../GATNN/embeddings/test_embeddings.pt ======
✅File hợp lệ!
Kích thước: (801, 512)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA FILE: ../GATNN/embeddings/train_embeddings.pt ======
✅File hợp lệ!
Kích thước: (6404, 512)___Kiểu dữ liệu: torch.float32___Có IDs không? Có
KIỂM TRA FIL

In [8]:
bert_all = torch.load(BERT_EMBEDDING, weights_only=False)
gat_all = torch.load(GNN_EMBEDDING, weights_only=False)

In [33]:
print('bert_all:')
print(type(bert_all))
print(bert_all.keys())
print('gat_all:')
print(type(gat_all))
print(gat_all.keys())

bert_all:
<class 'dict'>
dict_keys(['embeddings', 'mol_id', 'labels'])
gat_all:
<class 'dict'>
dict_keys(['embeddings', 'logits', 'probabilities', 'labels', 'mol_ids', 'num_graphs', 'embedding_dim', 'num_classes'])


In [19]:
print("Số lượng mẫu BERT:", len(bert_all['mol_id']))
print("Số lượng mẫu GAT:", len(gat_all['mol_ids']))

print("\n5 ID đầu tiên của BERT:", bert_all['mol_id'][:5])
print("5 ID đầu tiên của GAT:", gat_all['mol_ids'][:5])


bert_ids = set(bert_all['mol_id'])
gat_ids = set(gat_all['mol_ids'])

intersection_ids = bert_ids.intersection(gat_ids)
missing_in_gat = bert_ids - gat_ids
missing_in_bert = gat_ids - bert_ids

print("Tổng số ID trong BERT:", len(bert_ids))
print("Tổng số ID trong GAT:", len(gat_ids))
print("Số ID giao nhau:", len(intersection_ids))
print("Số ID BERT có nhưng GAT không có:", len(missing_in_gat))
print("Số ID GAT có nhưng BERT không có:", len(missing_in_bert))



Số lượng mẫu BERT: 8014
Số lượng mẫu GAT: 8006

5 ID đầu tiên của BERT: ['TOX31644', 'TOX7580', 'TOX24399', 'TOX5307', 'TOX26872']
5 ID đầu tiên của GAT: ['TOX13161', 'TOX29296', 'TOX4629', 'TOX213', 'TOX25950']
Tổng số ID trong BERT: 8014
Tổng số ID trong GAT: 8006
Số ID giao nhau: 8006
Số ID BERT có nhưng GAT không có: 8
Số ID GAT có nhưng BERT không có: 0


In [20]:
def check_alignment(bert_data, gat_data, split_name):
    """Kiểm tra xem mol_ids có khớp nhau không"""
    bert_ids = bert_data['mol_id']  
    gat_ids = gat_data['mol_ids']
    
    # Chuyển thành list thuần (phòng trường hợp tensor)
    if not isinstance(bert_ids, list):
        bert_ids = list(bert_ids)
    if not isinstance(gat_ids, list):
        gat_ids = list(gat_ids)
    
    print(f"Số lượng ID BERT: {len(bert_ids)}")
    print(f"Số lượng ID GAT : {len(gat_ids)}")

    # KTRA SỐ LƯỢNG

    # So sánh
    if len(bert_ids) != len(gat_ids):
        print(f" {split_name}: Số lượng không khớp! BERT={len(bert_ids)}, GAT={len(gat_ids)}")
    
    #Tìm các ID có trong BERT nhưng không có trong GAT
    missing_in_gat = set(bert_ids) - set(gat_ids)
    missing_in_bert = set(gat_ids) - set(bert_ids)

    print('KIỂM TRA PHẦN TỬ BỊ THIẾU:')

    if len(missing_in_gat) == 0:
        print("Không có ID nào trong BERT bị thiếu ở GAT")
    else:
        print("ID có trong BERT nhưng KHÔNG có trong GAT:")
        print(list(missing_in_gat))

    if len(missing_in_bert) == 0:
        print("Không có ID nào trong GAT bị thiếu ở BERT")
    else:
        print("ID có trong GAT nhưng KHÔNG có trong BERT:")
        print(list(missing_in_bert))

    # Nếu số lượng khớp hoàn toàn => kiểm tra thứ tự
    # Kiểm tra thứ tự
    if len(bert_ids) == len(gat_ids):
        print("\nKIỂM TRA THỨ TỰ PHẦN TỬ:")
        same_order = all(b == g for b, g in zip(bert_ids, gat_ids))

        if same_order:
            print("THỨ TỰ GIỐNG NHAU!")
        else:
            print("Thứ tự KHÔNG khớp!")
            print("5 phần tử đầu:")
            for i in range(5):
                print(f"  BERT[{i}] = {bert_ids[i]},  GAT[{i}] = {gat_ids[i]}")
    else:
        print("Bỏ qua kiểm tra thứ tự vì số lượng không khớp")

# Check tất cả splits
# train_match = check_alignment(bert_train, gat_train, "Train")
# val_match = check_alignment(bert_val, gat_val, "Val")
# test_match = check_alignment(bert_test, gat_test, "Test")
all_match = check_alignment(bert_all, gat_all, "All Data")


# output 
# số lượng k = nhau nên k thể align theo thứ tự 

Số lượng ID BERT: 8014
Số lượng ID GAT : 8006
 All Data: Số lượng không khớp! BERT=8014, GAT=8006
KIỂM TRA PHẦN TỬ BỊ THIẾU:
ID có trong BERT nhưng KHÔNG có trong GAT:
['TOX28623', 'TOX24552', 'TOX24723', 'TOX24622', 'TOX31563', 'TOX24724', 'TOX7518', 'TOX28892']
Không có ID nào trong GAT bị thiếu ở BERT
Bỏ qua kiểm tra thứ tự vì số lượng không khớp


In [43]:
bert_all['labels']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [46]:
# đảm bảo bert gat cùng số mẫu 
# đúng thứ tự theo mol id 
def align_datasets(bert_data, gat_data):
    """Align BERT và GAT theo mol_id (không kiểm tra labels)."""

    bert_ids = bert_data['mol_id']
    gat_ids  = gat_data['mol_ids']

    # Tạo dictionary để tìm index nhanh
    bert_dict = {str(m): i for i, m in enumerate(bert_ids)}
    gat_dict  = {str(m): i for i, m in enumerate(gat_ids)}

    # Lấy phần giao của mol_id
    common_ids = sorted(set(bert_dict.keys()) & set(gat_dict.keys()))

    print(f"BERT có {len(bert_ids)} mẫu")
    print(f"GAT  có {len(gat_ids)} mẫu")
    print(f"Trùng mol_id: {len(common_ids)} mẫu")

    bert_indices = [bert_dict[m] for m in common_ids]
    gat_indices  = [gat_dict[m] for m in common_ids]

    # Embeddings
    bert_embs = bert_data["embeddings"][bert_indices]
    gat_embs  = gat_data["embeddings"][gat_indices]

    # Labels
    bert_labels = bert_data["labels"][bert_indices]
    gat_labels  = gat_data["labels"][gat_indices]

    aligned_bert = {
        "embeddings": bert_embs,
        "labels": bert_labels,
        "mol_id": common_ids
    }

    aligned_gat = {
        "embeddings": gat_embs,
        "labels": gat_labels,
        "mol_ids": common_ids
    }

    return aligned_bert, aligned_gat


# Align từng split

all_bert_aligned, all_gat_aligned = align_datasets(bert_all, gat_all)
# Tạo datasets


# print(f"\n Datasets created:")
# print(f"Train: {len(train_dataset)} samples")
# print(f"Val: {len(val_dataset)} samples")
# print(f"Test: {len(test_dataset)} samples")

BERT có 8014 mẫu
GAT  có 8006 mẫu
Trùng mol_id: 8006 mẫu


In [41]:
from torch.utils.data import Dataset, DataLoader

class FusionDataset(Dataset):
    def __init__(self, bert_data, gat_data):
        # Embeddings
        self.text_embs = bert_data['embeddings']
        self.graph_embs = gat_data['embeddings']
        
        # Labels (lấy từ 1 trong 2, giả sử giống nhau)
        self.labels = bert_data['labels']
        
        # Optional: mol_ids để track
        self.mol_ids = bert_data['mol_id']
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'text_emb': self.text_embs[idx],
            'graph_emb': self.graph_embs[idx],
            'label': self.labels[idx],
            'mol_id': self.mol_ids[idx]
        }



In [52]:
import numpy as np
from sklearn.model_selection import train_test_split

N = len(all_bert_aligned["mol_id"])
print("Total aligned samples:", N)
indices = np.arange(N)

print(type(all_bert_aligned["labels"]))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(test_idx, test_size=0.5, random_state=42)

print('len train index:',len(train_idx))
print('len val index:' , len(val_idx))
print('len test index:', len(test_idx))

Total aligned samples: 8006
<class 'torch.Tensor'>
len train index: 6404
len val index: 801
len test index: 801


In [53]:
def split_data(aligned_data, idx, mol_id):
    return {
        "embeddings": aligned_data["embeddings"][idx],
        "labels": aligned_data["labels"][idx],
        "mol_id": [aligned_data[mol_id][i] for i in idx]
    }
train_bert_aligned = split_data(all_bert_aligned, train_idx, "mol_id")
val_bert_aligned = split_data(all_bert_aligned, val_idx, "mol_id")
test_bert_aligned = split_data(all_bert_aligned, test_idx, "mol_id")

train_dataset = FusionDataset(train_bert_aligned, split_data(all_gat_aligned, train_idx, "mol_ids"))
val_dataset = FusionDataset(val_bert_aligned, split_data(all_gat_aligned, val_idx, "mol_ids"))
test_dataset = FusionDataset(test_bert_aligned, split_data(all_gat_aligned, test_idx, "mol_ids"))
print(f"\n Datasets created:")
print(f"Train: {len(train_dataset)} samples")
print(f"Val: {len(val_dataset)} samples")
print(f"Test: {len(test_dataset)} samples")


 Datasets created:
Train: 6404 samples
Val: 801 samples
Test: 801 samples


In [72]:
class GatedFusion(nn.Module):
    def __init__(self, 
                 dim_bert, 
                 dim_gat, 
                 common_dim=256, 
                 per_dim=True,
                 ):
        

        super().__init__()
        self.bert_to_common = nn.Linear(dim_bert, common_dim) #chiếu BERT sang không gian chung
        self.gat_to_common  = nn.Linear(dim_gat, common_dim)
        # Kích thước đầu vào của gate = concat(b,g) = 2 * common_dim
        gate_in_dim = common_dim * 2
        if per_dim:
            #alpha cho từng chiều của vector
            self.gate = nn.Sequential(
                nn.Linear(gate_in_dim, common_dim), # tính alpha cho từng chiều
                nn.Sigmoid() # đưa alpha về [0,1]
            )
        else:
            # alpha là 1 giá trị chung cho cả vector
            self.gate = nn.Sequential(
                nn.Linear(gate_in_dim, 1),
                nn.Sigmoid()
            )
        self.per_dim = per_dim

    def forward(self, bert_embedding, gat_embedding):
        # bert_x: (batch, dim_bert)
        b = torch.tanh(self.bert_to_common(bert_embedding))   # (batch, common_dim)
        g = torch.tanh(self.gat_to_common(gat_embedding))     # (batch, common_dim)
        # nối 2 vector trước khi vào gate  
        concat_features = torch.cat([b, g], dim=1)           # (batch, 2*common_dim)
        alpha = self.gate(concat_features)                   # (batch, common_dim) or (batch,1)
        # Fusion: weighted sum
        fused = alpha * b + (1.0 - alpha) * g   # broadcast nếu alpha scalar
    

        # trả về embedding cuối cùng sau train 
        return fused, alpha
    
class ToxicityModel(nn.Module):
    def __init__(self, dim_bert, dim_gat, num_labels=12):
        super().__init__()

        self.fusion = GatedFusion(dim_bert, dim_gat, common_dim=256, per_dim=True)

        self.classifier = nn.Sequential(
            nn.LayerNorm(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_labels)
        )

    def forward(self, bert_emb, gat_emb):
        fused, alpha = self.fusion(bert_emb, gat_emb)
        logits = self.classifier(fused)
        return logits, alpha
    



In [56]:

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Get dimensions
sample_batch = next(iter(train_loader))
text_dim = sample_batch['text_emb'].shape[1]  # 768
graph_dim = sample_batch['graph_emb'].shape[1]  # 512
num_labels = sample_batch['label'].shape[1]  # 12

print(f"\n=== Configuration ===")
print(f"Text dim: {text_dim}")
print(f"Graph dim: {graph_dim}")
print(f"Num labels (organs): {num_labels}")


# Initialize model (you can pass init_gate_pref to bias initial gate, and force_ratio to force global ratio)
model = ToxicityModel(
    dim_bert=text_dim,
    dim_gat=graph_dim,
    num_labels=num_labels
).to(device)


# Chạy này trước khi train để kiểm tra
batch = next(iter(train_loader))
print("=== Batch Debug ===")
print(f"text_emb: {batch['text_emb'].shape}, dtype: {batch['text_emb'].dtype}")
print(f"graph_emb: {batch['graph_emb'].shape}, dtype: {batch['graph_emb'].dtype}")
print(f"label: {batch['label'].shape}, dtype: {batch['label'].dtype}")
print(f"label values: {batch['label'][:5]}")  # First 5 labels



Using device: cpu

=== Configuration ===
Text dim: 768
Graph dim: 512
Num labels (organs): 12
=== Batch Debug ===
text_emb: torch.Size([32, 768]), dtype: torch.float32
graph_emb: torch.Size([32, 512]), dtype: torch.float32
label: torch.Size([32, 12]), dtype: torch.float32
label values: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [73]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# ========== DataLoaders ==========
# {'text_emb': Tensor[batch, 768], 'graph_emb': Tensor[batch, 512], 'label': Tensor[batch,12]}
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ========== Model ==========
# model = GatedFusion + classifier, output multi-label [batch,12]
model = model.to(device)

# ========== Loss & Optimizer ==========
all_labels = torch.cat([batch['label'] for batch in train_loader], dim=0)
label_freq = all_labels.sum(dim=0)
neg_freq = all_labels.size(0) - label_freq
pos_weight = (neg_freq / (label_freq + 1e-6)).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# ========== Training/Evaluation Functions ==========
def train_epoch(model, loader, optimizer, criterion, device, balance_alpha=0.0):
    model.train()
    total_loss = 0
    for batch in loader:
        text_emb = batch['text_emb'].to(device)
        graph_emb = batch['graph_emb'].to(device)
        labels   = batch['label'].to(device)  # [batch,12]

        # Forward
        logits, weights = model(text_emb, graph_emb)  # [batch,12], [batch,1,2]

        # BCE loss multi-label
        loss = criterion(logits, labels)

     
        if balance_alpha > 0:
            w = weights.squeeze(1)  # [batch,2]
            target = torch.ones_like(w) * 0.5
            balance_loss = ((w - target)**2).mean()
            loss = loss + balance_alpha * balance_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate_any_toxic(model, loader, device, threshold=0.5):
    model.eval()
    all_logits, all_labels, all_weights = [], [], []

    with torch.no_grad():
        for batch in loader:
            text_emb = batch['text_emb'].to(device)
            graph_emb = batch['graph_emb'].to(device)
            labels = batch['label'].to(device)  # [batch,12]

            logits, weights = model(text_emb, graph_emb)
            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())
            all_weights.append(weights.cpu())

    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)
    weights = torch.cat(all_weights, dim=0)

    # Any-toxic metrics
    labels_any = (labels.sum(dim=1) > 0).float()
    probs_any  = torch.sigmoid(logits).max(dim=1)[0]
    preds_any  = (probs_any > threshold).float()

    # Weight statistics
    w = weights.squeeze(1)
    bert_w = w[:,0]
    gat_w  = w[:,1]

    metrics = {
        'accuracy': accuracy_score(labels_any, preds_any),
        'f1': f1_score(labels_any, preds_any),
        'roc_auc': roc_auc_score(labels_any, probs_any),
        'avg_bert_weight': bert_w.mean().item(),
        'avg_gat_weight': gat_w.mean().item(),
        'bert_std': bert_w.std().item(),
        'gat_std': gat_w.std().item(),
        'mean_variance': ((w - 0.5)**2).mean().item()
    }
    return metrics

# ========== Training Loop ==========
best_val_f1 = 0
patience = 10
patience_counter = 0
balance_alpha = 0.03
min_variance_threshold = 0.001

for epoch in range(50):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device, balance_alpha)
    val_metrics = evaluate_any_toxic(model, val_loader, device)

    print(f"Epoch {epoch+1:2d} | Loss: {train_loss:.4f} | "
          f"F1: {val_metrics['f1']:.4f} | Acc: {val_metrics['accuracy']:.4f} | "
          f"BERT: {val_metrics['avg_bert_weight']:.3f} | GAT: {val_metrics['avg_gat_weight']:.3f} | "
          f"Var: {val_metrics['mean_variance']:.4f} | α={balance_alpha:.3f}")

    
    if val_metrics['mean_variance'] < min_variance_threshold and epoch > 5:
        balance_alpha = min(balance_alpha + 0.01, 0.05)
        print(f"  Variance too low! Increasing α to {balance_alpha:.3f}")


    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_metrics': val_metrics,
        }, 'best_any_toxic_model.pt')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

# ========== Test ==========
checkpoint = torch.load('best_any_toxic_model.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
test_metrics = evaluate_any_toxic(model, test_loader, device)

print("\n=== Test Results ===")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"F1: {test_metrics['f1']:.4f}")
print(f"ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"BERT avg weight: {test_metrics['avg_bert_weight']:.3f}")
print(f"GAT avg weight: {test_metrics['avg_gat_weight']:.3f}")
print(f"Weight variance: {test_metrics['mean_variance']:.4f}")


Using device: cpu
Epoch  1 | Loss: 0.0537 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.685 | GAT: 0.683 | Var: 0.0332 | α=0.030
Epoch  2 | Loss: 0.0422 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.664 | GAT: 0.707 | Var: 0.0294 | α=0.030
Epoch  3 | Loss: 0.0345 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.642 | GAT: 0.697 | Var: 0.0240 | α=0.030
Epoch  4 | Loss: 0.0289 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.615 | GAT: 0.682 | Var: 0.0204 | α=0.030
Epoch  5 | Loss: 0.0258 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.581 | GAT: 0.686 | Var: 0.0175 | α=0.030
Epoch  6 | Loss: 0.0249 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.575 | GAT: 0.671 | Var: 0.0145 | α=0.030
Epoch  7 | Loss: 0.0222 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.555 | GAT: 0.655 | Var: 0.0139 | α=0.030
Epoch  8 | Loss: 0.0182 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.565 | GAT: 0.660 | Var: 0.0124 | α=0.030
Epoch  9 | Loss: 0.0176 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.515 | GAT: 0.668 | Var: 0.0110 | α=0.030
Epoch 10 | Loss: 0.0164 | F1: 1.0000 | Acc: 1.0000 | BERT: 0.539

In [74]:
model.eval()
all_weights = []

for batch in val_loader:
    text_emb = batch['text_emb'].to(device)
    graph_emb = batch['graph_emb'].to(device)

    with torch.no_grad():
        _, attn = model(text_emb, graph_emb)
    
    # attn shape = [batch, 1, 2]  => squeeze thành [batch, 2]
    all_weights.append(attn.squeeze(1).cpu())

all_weights = torch.cat(all_weights, dim=0)

bert_contrib = all_weights[:, 0].mean().item()
gat_contrib  = all_weights[:, 1].mean().item()

print(" BERT contribution:", bert_contrib)
print(" GAT contribution :", gat_contrib)
print("Variance:", all_weights.var(dim=0))


 BERT contribution: 0.685248613357544
 GAT contribution : 0.6830386519432068
Variance: tensor([0.0047, 0.0034, 0.0013, 0.0024, 0.0028, 0.0039, 0.0035, 0.0010, 0.0024,
        0.0020, 0.0016, 0.0025, 0.0060, 0.0042, 0.0015, 0.0011, 0.0012, 0.0020,
        0.0036, 0.0010, 0.0015, 0.0024, 0.0026, 0.0019, 0.0035, 0.0019, 0.0024,
        0.0028, 0.0031, 0.0017, 0.0029, 0.0044, 0.0015, 0.0017, 0.0015, 0.0030,
        0.0027, 0.0022, 0.0023, 0.0017, 0.0021, 0.0034, 0.0022, 0.0041, 0.0021,
        0.0020, 0.0017, 0.0017, 0.0028, 0.0030, 0.0043, 0.0032, 0.0027, 0.0012,
        0.0038, 0.0023, 0.0017, 0.0040, 0.0029, 0.0022, 0.0007, 0.0018, 0.0030,
        0.0022, 0.0036, 0.0012, 0.0037, 0.0067, 0.0034, 0.0017, 0.0017, 0.0021,
        0.0028, 0.0016, 0.0020, 0.0013, 0.0018, 0.0028, 0.0032, 0.0036, 0.0033,
        0.0021, 0.0048, 0.0074, 0.0033, 0.0035, 0.0024, 0.0062, 0.0017, 0.0035,
        0.0022, 0.0013, 0.0024, 0.0031, 0.0023, 0.0019, 0.0033, 0.0011, 0.0029,
        0.0022, 0.0030, 0.0022, 0

In [79]:
def predict_from_embeddings(text_emb, graph_emb, model, device):
    model.eval()
    text_emb = text_emb.to(device)
    graph_emb = graph_emb.to(device)
    with torch.no_grad():
        logits, _ = model(text_emb, graph_emb)
        any_logits = logits.max(dim=1)[0]       # any-toxic
        prob = torch.sigmoid(any_logits)
        pred = (prob > 0.5).float()
    return pred.item(), prob.item()


idx = 10  # index của molecule

# Chuyển numpy -> tensor
text_emb = torch.tensor(bert_all['embeddings'][idx], dtype=torch.float32).unsqueeze(0)  # [1, 768]
graph_emb = gat_all['embeddings'][idx].detach().clone().unsqueeze(0)

pred, prob = predict_from_embeddings(text_emb, graph_emb, model, device)
if pred == 1:
    print('Dự đoán SMILE toxic')
elif pred == 0:
    print('Dự đoán SMILE non toxic')

print(f"Probability: {prob:.3f}")



Dự đoán SMILE toxic
Probability: 0.997
